In [4]:
import math
import sys
from collections import namedtuple, deque
from pprint import pprint as pp

In [28]:
#DIT374 Assignment 7
#Erik Rosvall 960523
#Christoffer Wikner 931012

#Question 1

class Graph:
    def __init__(self, gdict = {}):
        self.gdict = gdict
        self.colour = {}
        self.distance = {}
        self.predecessor = {}
        self.finish = {}     
        
    def get_vertices(self):
        #return list(self.gdict.keys())
        vertices = set()
        for u in self.gdict:
            vertices.add(u)
            for v in self.gdict[u]:
                vertices.add(v)
        return vertices
    
    def get_edges(self):
        edgename = []
        for vertex in self.gdict:
            for next_vertex in self.gdict[vertex]:
                if {next_vertex, vertex} not in edgename:
                    edgename.append({vertex, next_vertex})
        return edgename    
    
    def add_vertex(self, vertex):
        if vertex not in self.gdict:
            self.gdict[vertex] = []
            
    def add_edge(self, edge):
        edge = set(edge)
        (vertex1, vertex2) = tuple(edge)
        if vertex1 in self.gdict:
            self.gdict[vertex1].append(vertex2)
        else:
            self.gdict[vertex1] = [vertex2]
            
    # BFS and DFS are implemented using the pseudo-codes from the Book
    def BFS(self, s):
        vertices = self.get_vertices()
        vertices.remove(s)
        for u in vertices:
            self.colour[u] = "White"
            self.distance[u] = 0
            self.predecessor[u] = None
        self.colour[s] = "Gray"
        self.distance[s] = 0
        self.predecessor[s] = None
        queue = []
        queue.append(s)
        while queue:
            u = queue.pop(0)
            #print(self.gdict[u])
            for v in self.gdict[u]:
                if self.colour[v] == "White":
                    self.colour[v] = "Gray"
                    self.distance[v] = self.distance[u] + 1
                    self.predecessor[v] = u
                    queue.append(v)
            self.colour[u] = "Black"    
            
    def print_path(self, s, v):
        if v in self.gdict.keys():
            if v == s:
                print(s)
            elif self.predecessor[v] == None:
                print("There is no path from", s, "to", v, "exists.")
            else:
                self.print_path(s, self.predecessor[v])
                print(v)
        else: 
            print("Node with key", v, "is not in the graph.")       
            
    def DFS(self):
        for u in self.gdict:
            self.colour[u] = "White"
            self.predecessor[u] = None
            self.distance[u] = 0    # Added to initialise self.distance
            self.finish[u] = -1     # Added to initialise self.finish
        global time 
        time = 0
        for u in self.gdict:
            if self.colour[u] == "White":
                self._DFS_visit(u)        
                
    def _DFS_visit(self, u):
        global time
        time += 1      # white vertex u has just been discovered
        self.distance[u] = time
        self.colour[u] = "Gray"
        for v in self.gdict[u]:  # explore edge(u,v)
            if self.colour[v] == "White":
                self.predecessor[v] = u
                self._DFS_visit(v)
        self.colour[u] = "Black"   # blacken u; it is finished
        time += 1
        self.finish[u] = time

    # ----- New functionality -----

    def initialise_single_source(self, s):
        for v in self.get_vertices():
            self.distance[v] = math.inf # close to infinity
            self.predecessor = None
        self.distance[s] = 0

    def get_weight(self, u, v):
        return self.gdict[u][v]

    def relax(self, v, u):
        if self.distance[v] > self.distance[u] + self.get_weight(u,v):
            self.distance[v] = self.distance[u] + self.get_weight(u, v)
            self.predecessor[v] = u

    # DAG
    def dag_shortest_paths(self, s):
        self.topological_sort()  # O(V+E)
        self.initialise_single_source(s)  # O(V)
        for u in self.finish.keys():      # O(V+E)
            for v in self.gdict[u].keys():
                self.relax(u, v)
    
    def extract_min(self, candidates):
        min = 999999
        for cu in candidates:
            if self.distance[cu] < min:
                min = self.distance[cu]
                u = cu  
        return u

    def topological_sort(self):
        self.DFS()
        #return self.finish()


    
    # Dijkstra Algorithm
    def dijkstra(self, s):
        #pri = pq()
        self.initialise_single_source(s)
        # S is a list for vertices that have been processed
        S = []
        # Q is a list for vertices that have not been processed
        Q = self.get_vertices() 
        while Q:
            u = self.extract_min(Q)
            Q.remove(u)
            S = S + [u] 
            #pri.insert(u)
            for v in self.gdict[u]:
                self.relax(u, v)
        print(S)

In [29]:
class pq(object):
    def __init__(self):
        self.queue = []
  
    def __str__(self):
        return ' '.join([str(i) for i in self.queue])
  
    # for checking if the queue is empty
    def isEmpty(self):
        return len(self.queue) == 0
  
    # for inserting an element in the queue
    def insert(self, data):
        self.queue.append(data)
  
    # for popping an element based on Priority
    def delete(self):
        try:
            max = 0
            for i in range(len(self.queue)):
                if self.queue[i] > self.queue[max]:
                    max = i
            item = self.queue[max]
            del self.queue[max]
            return item
        except IndexError:
            print()
            exit()

In [30]:
def dijkstra(self, source, dest):
        assert source in self.get_vertices()
        self.initilise_single_source(source)
        dist = {vertex: float('inf') for vertex in self.get_vertices()}
        self.predecessor = {vertex: None for vertex in self.get_vertices()}
        dist[source] = 0 # Tror den kommer ge fel
        q = self.get_vertices()
        neighbours = {vertex: set() for vertex in self.get_vertices()}
        """
        for start, end, cost in self.get_edges():
            neighbours[start].add((end, cost))
            neighbours[end].add((start, cost))
        """
        pp(neighbours)
 
        while q:
            #pp(q)
            u = min(q, key=lambda vertex: dist[vertex])
            q.remove(u)
            if dist[u] == float('inf') or u == dest:
                break
            for v, cost in neighbours[u]:
                alt = dist[u] + cost
                if alt < dist[v]: # Relax (u,v,a)
                    dist[v] = alt
                    previous[v] = u
                #self.relax(v, u)
        #pp(previous)
        pp(dest) # printar
        s, u = deque(), dest
        cc = self.predecessor
        pp(cc) # printar
        while cc[u]:
            s.appendleft(u)
            u = self.predecessor[u]
        s.appendleft(u)
        return s


In [33]:
#g5 = (Graph(adjacency5))

adjacency5 = {"s" : {"t": 10, "y": 5},
              "t" : {"x": 1, "y": 2},
              "x" : {"z": 4},
              "y" : {"t": 3, "x": 9, "z": 2},
              "z" : {"x": 6, "s": 7}
              }

gg = Graph(adjacency5)

In [34]:
g5.dijkstra("s")

KeyError: 's'